In [108]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn


def loc(nb):
    cells = load(open(nb))['cells']
    return sum(len(c['source']) for c in cells if c['cell_type'] == 'code')

def run(ipynb_files):
    return sum(loc(nb) for nb in ipynb_files)

# the following neural network is to model the value function
class MyNet(nn.Module):
    def __init__(self, m, n):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(1 + m * n, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU(inplace=False)

# x is 2 dim torch tensor with shape (m, n)
    def forward(self, t, x):
        #make x 1 dim
        x = x.view(-1)
        x = torch.cat((t, x), dim=0)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    
# The Gaussian policy returns torch tensor with shape (m, n)
def Gaussian_Policy(net, t, q, A, B, gamma):
    
    bid_matrix  = torch.zeros(q.shape)
    ask_matrix  = torch.zeros(q.shape)

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            x = torch.zeros(q.shape)
            x[i, j] = 1
            mean_bid = A[i, j] / (2 * B[i, j]) + 0.5 * (net.forward(t, q) - net.forward(t, q + x))
            mean_ask = A[i, j] / (2 * B[i, j]) + 0.5 * (net.forward(t, q) - net.forward(t, q - x))
            variance = gamma / (2 * B[i, j])
            std = torch.sqrt(variance)
            bid_matrix[i, j] = torch.normal(mean_bid, std)
            ask_matrix[i, j] = torch.normal(mean_ask, std)

    return bid_matrix, ask_matrix
    

import math
import scipy.stats as scistat

def Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix):
    orders = torch.zeros(bid_matrix.shape)
    buy_orders = torch.zeros(bid_matrix.shape)
    sell_orders = torch.zeros(bid_matrix.shape)
    m = bid_matrix.shape[0]
    n = bid_matrix.shape[1]
    for i in range(m):
        for j in range(n):
            intensity_bid = A[i, j] - B[i, j] * bid_matrix[i, j]
            intensity_ask = A[i, j] - B[i, j] * ask_matrix[i, j]
            buy_orders[i, j] = torch.poisson(intensity_bid * dt)
            sell_orders[i, j] = torch.poisson(intensity_ask * dt)
            orders[i, j] = buy_orders[i, j] - sell_orders[i, j]

    return [orders, buy_orders, sell_orders]

# this function returns the inventory path
def Train_Data_Simulation(T, dt, A, B, gamma, net):
    N = int(T / dt)
    m = A.shape[0]
    n = A.shape[1]
    q = torch.zeros((N, m, n))
    buy = torch.zeros((N, m, n))
    sell = torch.zeros((N, m, n))
    bid = torch.zeros((N, m, n))
    ask = torch.zeros((N, m, n))

    for count in range(N - 1):
        t = torch.Tensor([count * dt])
        bid_matrix, ask_matrix = Gaussian_Policy(net, t, q[count], A, B, gamma)
        bid[count] = bid_matrix
        ask[count] = ask_matrix
        orders = Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix)[0]
        buy[count] = Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix)[1]
        sell[count] = Market_Order_Simulation(dt, A, B, bid_matrix, ask_matrix)[2]
        q[count + 1] = q[count] + orders
    
    return q, buy, sell, bid, ask

# this function returns the stock prices path
def Stock_Prices_Simulation(T, dt, mu, sigma, S0):
    N = int(T / dt)
    S = torch.zeros(N)
    S[0] = S0
    for count in range(N - 1):
        S[count + 1] = S[count] + mu * S[count] * dt + sigma * S[count] * math.sqrt(dt) * torch.normal(0.0, 1.0, size=(1,))
    return S


#the following are option greeks' functions
def d1(S, K, r, sigma, T):
    return (np.log(S/K) + (r+sigma*sigma/2)*T)/(sigma*np.sqrt(T))

def d2(S, K, r, sigma, T):
    return d1(S, K, r, sigma, T) - sigma*np.sqrt(T)

def Gamma(S, K, r, sigma, T):
    return scistat.norm.pdf(d1(S, K, r, sigma, T))/(S*sigma*np.sqrt(T))

def Theta(S, K, r, sigma, T):
    aux1 = -S*scistat.norm.pdf(d1(S, K, r, sigma, T))*sigma/(2*np.sqrt(T))
    aux2 = -r*K*np.exp(-r*T)*scistat.norm.cdf(d2(S, K, r, sigma, T))
    return aux1+aux2


def Options_Theta(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes):
    N = int(T / dt)
    m = Vol_surface.shape[0]
    n = Vol_surface.shape[1]
    theta = torch.zeros((N, m, n))
    S = Stock_Prices_Simulation(T, dt, mu, sigma, S0)
    for count in range(N):
        t = count * dt
        for i in range(m):
            for j in range(n):
                theta[count, i, j] = Theta(S[count], Strikes[i], r, Vol_surface[i, j], Maturities[j] - t)
    return theta


def Options_Gamma(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes):
    N = int(T / dt)
    m = Vol_surface.shape[0]
    n = Vol_surface.shape[1]
    gamma = torch.zeros((N, m, n))
    S = Stock_Prices_Simulation(T, dt, mu, sigma, S0)
    for count in range(N):
        t = count * dt
        for i in range(m):
            for j in range(n):
                gamma[count, i, j] = Gamma(S[count], Strikes[i], r, Vol_surface[i, j], Maturities[j] - t)
    return gamma


In [109]:
# define the reward function in the following
# buy, sell are of shape(N, m,n)
# q is of shape(N, m, n)
def reward(dt, A, B, gamma, net, T, S0, r, mu, sigma, Maturities, Strikes, q, buy, sell, Vol_surface):
    N = int(T / dt)
    m = A.shape[0]
    n = A.shape[1]
    S = Stock_Prices_Simulation(T, dt, mu, sigma, S0)
    reward = torch.zeros(N)
    for count in range(N):
        t = torch.tensor([count * dt])
        reward[count] = torch.tensor([0.0])
        for i in range(m):
            for j in range(n):
                # declare a 2 dim tensor of shape (m,n)
                x = torch.zeros(m, n)
                x[i, j] = 1
                reward[count] = reward[count] + (buy[count, i, j] * ((A[i,j] / (2 * B[i,j])) + 0.5 * (net.forward(t, q[count]) - net.forward(t, q[count] + x)))) * dt
                reward[count] = reward[count] + (sell[count, i, j] * ((A[i,j] / (2 * B[i,j])) + 0.5 * (net.forward(t, q[count]) - net.forward(t, q[count] - x)))) * dt

        reward[count] += torch.sum((Options_Theta(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes)[count] + 0.5 * Options_Gamma(Vol_surface, S0, r, mu, sigma, T, dt, Maturities, Strikes)[count]) * q[count]) * dt
        reward[count] = reward[count] - gamma * ((m * n * 1.79817986835) + torch.sum(torch.log(gamma / (2 * B)))) * dt
    
    return torch.sum(reward)

    

In [110]:
def training_data_collections(D, T, dt, A, B, gamma, net):
    N = int(T / dt)
    m = A.shape[0]
    n = A.shape[1]
    train_q = torch.zeros((D, N, m, n))
    train_buy = torch.zeros((D, N, m, n))
    train_sell = torch.zeros((D, N, m, n))
    bid = torch.zeros((D, N, m, n))
    ask = torch.zeros((D, N, m, n))
    for count in range(D):
        train_q[count], train_buy[count], train_sell[count], bid[count], ask[count] = Train_Data_Simulation(T, dt, A, B, gamma, net)
    return train_q, train_buy, train_sell, bid, ask

def train_stock(D, T, dt, mu, sigma, S0):
    N = int(T / dt)
    train_data = torch.zeros((D, N))
    for count in range(D):
        train_data[count] = Stock_Prices_Simulation(T, dt, mu, sigma, S0)
    return train_data



In [122]:
import torch.distributions as dist
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal

def kl_divergence(new_net, old_net, T, dt, A, B, gamma, q):
    m = A.shape[0]
    n = A.shape[1]
    N = int(T / dt)
    kl = torch.zeros(N)
    old_mean = torch.zeros((N, 2 * m * n))
    new_mean = torch.zeros((N, 2 * m * n))
    cov = torch.zeros((N, 2 * m * n, 2 * m * n))
    for count in range(N):
        t = torch.tensor([count * dt])
        for i in range(m):
            for j in range(n):
                # declare a 2 dim tensor of shape (m,n)
                x = torch.zeros(m, n)
                x[i, j] = 1
                old_mean[count, i * n + j] = A[i, j] / (2 * B[i, j]) + 0.5 * (old_net.forward(t, q[count]) - old_net.forward(t, q[count] + x))
                old_mean[count, i * n + j + 1] = A[i, j] / (2 * B[i, j]) + 0.5 * (old_net.forward(t, q[count]) - old_net.forward(t, q[count] - x))
                new_mean[count, i * n + j] = A[i, j] / (2 * B[i, j]) + 0.5 * (new_net.forward(t, q[count]) - new_net.forward(t, q[count] + x))
                new_mean[count, i * n + j + 1] = A[i, j] / (2 * B[i, j]) + 0.5 * (new_net.forward(t, q[count]) - new_net.forward(t, q[count] - x))
                cov[count, i * n + j, i * n + j] = gamma / (2 * B[i, j])
                cov[count, m * n + i * n + j, m * n + i * n + j] = gamma / (2 * B[i, j])

        old_dist = MultivariateNormal(old_mean[count], cov[count])
        new_dist = MultivariateNormal(new_mean[count], cov[count])
        kl[count] = dist.kl_divergence(old_dist, new_dist)
    return torch.sum(kl)



# q, bid, ask is of shape (N, m,n)
def normal_distribution_pdf(T, dt, A, B, gamma, net, q, bid, ask):
    m = A.shape[0]
    n = A.shape[1]
    N = int(T / dt)
    pdf_bid = torch.ones(N)
    pdf_ask = torch.ones(N)
    for count in range(N):
        t = torch.tensor([count * dt])
        for i in range(m):
            for j in range(n):
                # declare a 2 dim tensor of shape (m,n)
                x = torch.zeros(m, n)
                x[i, j] = 1
                mean_bid = A[i, j] / (2 * B[i ,j]) + 0.5 * (net.forward(t, q[count]) - net.forward(t, q[count] + x))
                mean_ask = A[i, j] / (2 * B[i ,j]) + 0.5 * (net.forward(t, q[count]) - net.forward(t, q[count] - x))
                cov = gamma / (2 * B[i, j])
                std = torch.sqrt(cov)
                pdf_bid[count] = pdf_bid[count] * torch.exp(Normal(mean_bid, std).log_prob(bid[count, i, j]))
                pdf_ask[count] = pdf_ask[count] * torch.exp(Normal(mean_ask, std).log_prob(ask[count, i, j]))

    pdf = pdf_bid * pdf_ask
    return pdf
        
    

def total_loss(dt, A, B, gamma, new_net, old_net, T, S0, r, mu, sigma, Maturities, Strikes, Vol_surface, beta, train_q, train_buy, train_sell, bid, ask, D):
    loss = torch.tensor([0.0])

    for count in range(D):
        pdf_new = normal_distribution_pdf(T, dt, A, B, gamma, new_net, train_q[count], bid[count], ask[count])
        pdf_old = normal_distribution_pdf(T, dt, A, B, gamma, old_net, train_q[count], bid[count], ask[count])
        loss = loss + reward(dt, A, B, gamma, old_net, T, S0, r, mu, sigma, Maturities, Strikes, train_q[count], train_buy[count], train_sell[count], Vol_surface) * torch.sum((pdf_new / pdf_old)[:-1])
        loss = loss - beta * kl_divergence(new_net, old_net, T, dt, A, B, gamma, train_q[count])

    return loss / D 



    


In [112]:
# check the reward function
Vol_surface = torch.tensor([[0.2, 0.3], [0.4, 0.5]])
S0 = 100
r = 0.05
T = 1
dt = 0.01
mu = 0.1
sigma = 0.2
gamma = 0.05
Maturities = torch.tensor([3, 5])
Strikes = torch.tensor([90, 110])
net = MyNet(2, 2)
learning_rate = 0.01
num_epochs = 30
A = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
B = torch.tensor([[0.2, 0.3], [0.2, 0.5]])
learning_rate = 0.01
num_epochs = 30
D = 2

beta = 0.1
threshold = 0.01



In [113]:
new_net = MyNet(2, 2)
old_net = MyNet(2, 2)




In [125]:
for i in range(num_epochs):
    torch.autograd.set_detect_anomaly(True)
    print(i)
    old_net.load_state_dict(new_net.state_dict())
    train_q, train_buy, train_sell, bid, ask = training_data_collections(D, T, dt, A, B, gamma, old_net)
    train_S = train_stock(D, T, dt, mu, sigma, S0)
    optimizer = torch.optim.Adam(new_net.parameters(), lr=learning_rate)
    optimizer.zero_grad()
    loss = total_loss(dt, A, B, gamma, new_net, old_net, T, S0, r, mu, sigma, Maturities, Strikes, Vol_surface, beta, train_q, train_buy, train_sell, bid, ask, D)
    loss.backward()
    optimizer.step()

    
    if kl_divergence(new_net, old_net, T, dt, A, B, gamma, train_q) > 1.5 * threshold:
        beta = beta * 2
    elif kl_divergence(new_net, old_net, T, dt, A, B, gamma, train_q) < threshold / 1.5:
        beta = beta / 2



0


KeyboardInterrupt: 

In [115]:
torch.autograd.set_detect_anomaly(True)